## Análisis para Generar Rutas

### Preprocesamiento

In [3]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
#import numpy as np
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer


###### ArCo ARMADO DE LA BASE#######
import os
from typing import List
from datetime import datetime

# Establecer el directorio de trabajo - DNP
os.chdir("C:/Users/supegui/Documents/Metodología ArCo/Python/Excel")

# Establecer el directorio de trabajo - Remoto
#os.chdir("C:/Backup - Archivos de trabajo/DNP/Automatización ArCo/Excel")


# Saca la lista de archivos del directorio de trabajo
#os.listdir()
# Obtener la fecha actual
fecha_actual = datetime.now().strftime("%Y %m %d")
#print(fecha_actual)

# Construir el nombre del archivo basado en la fecha actual
#nombre_archivo = f"{fecha_actual} Informe ArCo.xlsx"

# Construir el nombre del archivo cuando la fecha no es la actual. Aquí solo se modifica la fecha. ArCo Nal
#nombre_archivo = "2024 11 01 - Base Original Desc Completa.xlsx"

# Construir el nombre del archivo cuando la fecha no es la actual. Aquí solo se modifica la fecha. ArCo Regional
nombre_archivo = "2024 11 07 - Base ArCo Regional Original.xlsx"

data_arco = pd.read_excel(nombre_archivo)

In [4]:
data_arco.head()

,Código,Sector,Entidad,Nombre del Instrumento,Descripción,Página Web,Años Existencia_x,Iteración,Emprendedores,Microempresa,...,Educación económica y financiera,Competencias Digitales,Si el instrumento es de propiedad intelectual por favor marque aquí!,¿Qué tipos de derechos de propiedad intelectual contempla?,Recursos fuente PGN,Recursos fuente SGR,Recursos fuente Cooperacion,Recursos fuente Esfuerzo Fiscal,Recursos fuente Parafiscales,Recursos fuente Otro
0,3261,Comercio,COLOMBIA PRODUCTIVA,Fábricas de Productividad y Sostenibilidad - H...,Fábricas de Productividad y Sostenibilidad es ...,https://www.fabricasdeproductividadysostenibil...,5,NaN,No,Si,...,No,No,NaN,No,0,0,0,0,0,0
1,3304,NaN,Politécnico Grancolombiano,Cursos virtuales de Emprendimiento,"El Politécnico Grancolombiano, institución líd...",https://www.poli.edu.co/noticias/cursos-moocs-...,3,NaN,Si,No,...,Si,Si,NaN,No,0,0,0,0,0,0
2,3308,NaN,Camacol Huila,Expovivienda,Es la mejor feria de la construcción de Vivien...,https://www.camacolhuila.com/eventos/ferias/,10,NaN,Si,No,...,No,No,NaN,No,0,0,0,0,0,0
3,3321,Comercio,INNPULSA COLOMBIA,CEmprende,Conforme a la visión de cambio del Gobierno Na...,https://www.innpulsacolombia.com/cemprende/,3,NaN,No,Si,...,No,No,NaN,No,0,0,0,0,0,0
4,3403,NaN,Alcaldía Municipal de La Plata Huila,"VERSIÓN 184 DE LA FERIA GANADERA, MERCADO GANA...",Realizar de la VERSIÓN 184 DE LA FERIA GANADER...,http://www.laplata-huila.gov.co/,50,Primera Iteración 2023 Regional - Junio 2023,No,Si,...,No,No,NaN,No,0,0,200000000,0,0,0


In [5]:
# Get data information
data_arco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 63 columns):
 #   Column                                                                Non-Null Count  Dtype  
---  ------                                                                --------------  -----  
 0   Código                                                                13 non-null     int64  
 1   Sector                                                                2 non-null      object 
 2   Entidad                                                               13 non-null     object 
 3   Nombre del Instrumento                                                13 non-null     object 
 4   Descripción                                                           13 non-null     object 
 5   Página Web                                                            13 non-null     object 
 6   Años Existencia_x                                                     13 non-null     int64  
 7   I

In [33]:
# Selecting the relevant features for recommendation - Usuarios completos
selected_features = ['Emprendedores',
                 'Microempresa',
                 'Pequeña Empresa',
                 'Mediana Empresa',   
                 #'Mipymes',
                 #'Grandes empresas',
                 'Gobierno ',
                 'Academia',
                 #'Entidades de soporte',
                 'Personas naturales',
                 ]

print(selected_features)

"""
# Selecting the relevant features for recommendation
selected_features = [
                 #'Emprendedores',
                 'Mipymes',
                 'Grandes empresas',
                 #'Gobierno ',
                 #'Academia',
                 #'Entidades de soporte',
                 #'Personas naturales'
                 ]
print(selected_features)
"""

['Emprendedores', 'Microempresa', 'Pequeña Empresa', 'Mediana Empresa', 'Gobierno ', 'Academia', 'Personas naturales']


"\n# Selecting the relevant features for recommendation\nselected_features = [\n                 #'Emprendedores',\n                 'Mipymes',\n                 'Grandes empresas',\n                 #'Gobierno ',\n                 #'Academia',\n                 #'Entidades de soporte',\n                 #'Personas naturales'\n                 ]\nprint(selected_features)\n"

In [34]:
# Replacing the null valuess with null string
for feature in selected_features:
    data_arco[feature] = data_arco[feature].fillna('')

In [35]:
# combining all the 5 selected features - Con todos los tipos de usuarios
combined_features = data_arco['Emprendedores'] + ' ' + data_arco['Microempresa'] + ' ' + data_arco['Pequeña Empresa'] + ' ' + data_arco['Mediana Empresa'] + ' ' + data_arco['Gobierno '] + ' ' + data_arco['Academia'] + ' ' + data_arco['Personas naturales'] 

combined_features



0     No Si Si Si Si Si No
1     Si No No No No No Si
2     Si No No No No No Si
3     No Si Si Si Si Si Si
4     No Si Si Si Si Si Si
5     No Si Si Si Si Si Si
6     Si No No No No No Si
7     Si Si Si Si Si Si Si
8     No No No No No No Si
9     No Si Si Si Si Si Si
10    Si No No No Si No Si
11    No Si Si Si Si Si No
12    Si Si Si Si Si Si Si
dtype: object

### Construyendo el sistema de recomendación con el nombre del instrumento

In [36]:
# converting the text data to feature vectors
vectorizer = TfidfVectorizer()

feature_vectors = vectorizer.fit_transform(combined_features)

In [37]:
print(feature_vectors)

  (np.int32(0), np.int32(0))	0.4191493477680171
  (np.int32(0), np.int32(1))	0.907917300344941
  (np.int32(1), np.int32(0))	0.944862941904726
  (np.int32(1), np.int32(1))	0.3274660608599715
  (np.int32(2), np.int32(0))	0.944862941904726
  (np.int32(2), np.int32(1))	0.3274660608599715
  (np.int32(3), np.int32(0))	0.18889545655167372
  (np.int32(3), np.int32(1))	0.9819972028952703
  (np.int32(4), np.int32(0))	0.18889545655167372
  (np.int32(4), np.int32(1))	0.9819972028952703
  (np.int32(5), np.int32(0))	0.18889545655167372
  (np.int32(5), np.int32(1))	0.9819972028952703
  (np.int32(6), np.int32(0))	0.944862941904726
  (np.int32(6), np.int32(1))	0.3274660608599715
  (np.int32(7), np.int32(1))	1.0
  (np.int32(8), np.int32(0))	0.9897336933595042
  (np.int32(8), np.int32(1))	0.14292381267288798
  (np.int32(9), np.int32(0))	0.18889545655167372
  (np.int32(9), np.int32(1))	0.9819972028952703
  (np.int32(10), np.int32(0))	0.8385093221282828
  (np.int32(10), np.int32(1))	0.5448872513685448
  (n

In [38]:
# getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors, feature_vectors)

In [15]:
print(similarity)

[[1.         0.92446432 0.92446432 0.9305387  0.9305387  0.9305387
  0.92446432 0.79254622 0.76377229 0.9305387  0.92446432 1.
  0.79254622]
 [0.92446432 1.         1.         0.72063173 0.72063173 0.72063173
  1.         0.50017862 0.95218367 0.72063173 1.         0.92446432
  0.50017862]
 [0.92446432 1.         1.         0.72063173 0.72063173 0.72063173
  1.         0.50017862 0.95218367 0.72063173 1.         0.92446432
  0.50017862]
 [0.9305387  0.72063173 0.72063173 1.         1.         1.
  0.72063173 0.96080411 0.47434693 1.         0.72063173 0.9305387
  0.96080411]
 [0.9305387  0.72063173 0.72063173 1.         1.         1.
  0.72063173 0.96080411 0.47434693 1.         0.72063173 0.9305387
  0.96080411]
 [0.9305387  0.72063173 0.72063173 1.         1.         1.
  0.72063173 0.96080411 0.47434693 1.         0.72063173 0.9305387
  0.96080411]
 [0.92446432 1.         1.         0.72063173 0.72063173 0.72063173
  1.         0.50017862 0.95218367 0.72063173 1.         0.92446432


In [44]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = data_arco['Nombre del Instrumento'].tolist()
print(list_of_all_titles)

['Fábricas de Productividad y Sostenibilidad - Huila', 'Cursos virtuales de Emprendimiento', 'Expovivienda', 'CEmprende', 'VERSIÓN 184 DE LA FERIA GANADERA, MERCADO GANADERO, MUESTRA GANADERA, CONCURSO DE LA VACA LECHERA, MUESTRA AGRO EMPRESARIAL, DE TURISMO Y MERCADO CAMP', 'FERIA EQUINA GRADO B EN SU VERSIÓN 185, MERCADO GANADERO, MUESTRA AGRO EMPRESARIAL, COMERCIAL DE TURISMO, MERCADO CAMPESINO EN EL MUNIC', 'Sistema de Información Turístico y Cultural de Huila', 'feria internacional de cafe cacao y agroturismo', 'CONVOCATORIA PARA EL SECTOR TURISMO DEL DEPARTAMENTO DEL HUILA.', 'CELEBRACIÓN DEL DÍA DEL CAMPESINO DEL MUNICIPIO DE LA PLATA HUILA', 'Convocatoria Concertación Proyectos y Estimulos Para Gestores Culturales Huilenses', 'Convocatoria', 'Gestión de la Información Estadística y Cartográfica "Sistema de Información Regional SIR-SIGDEGHU"']


In [57]:
# getting the movie name from the user
instrument_name = input(' Introducir el nombre del instrumento : ')
#instrument_name = 'Información estadística de la temática económica'


In [58]:
# finding the close match for the instrument name given by the user
find_close_match = difflib.get_close_matches(instrument_name, list_of_all_titles)
print(find_close_match)

['CEmprende']


In [59]:
from difflib import SequenceMatcher

def calculate_similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

# Ordenar los instrumentos según similitud con el nombre escogido
instrumentos_similares = sorted(
    list_of_all_titles,
    key=lambda x: calculate_similarity(instrument_name, x),
    reverse=True
)[:10]


In [60]:
print(instrumentos_similares)

['CEmprende', 'Expovivienda', 'Cursos virtuales de Emprendimiento', 'Convocatoria', 'feria internacional de cafe cacao y agroturismo', 'Convocatoria Concertación Proyectos y Estimulos Para Gestores Culturales Huilenses', 'Fábricas de Productividad y Sostenibilidad - Huila', 'Sistema de Información Turístico y Cultural de Huila', 'Gestión de la Información Estadística y Cartográfica "Sistema de Información Regional SIR-SIGDEGHU"', 'CONVOCATORIA PARA EL SECTOR TURISMO DEL DEPARTAMENTO DEL HUILA.']


### Análisis a partir de descripciones y features

In [16]:
# Fill missing values with empty strings
data_arco['Descripción'] = data_arco['Descripción'].fillna('')

# Optionally, convert text to lowercase
data_arco['Descripción'] = data_arco['Descripción'].str.lower()

In [18]:
# Fill missing values with empty strings
data_arco['Descripción'] = data_arco['Descripción'].fillna('')

# Convert text to lowercase
data_arco['Descripción'] = data_arco['Descripción'].str.lower()

# Vectorize the descriptions
vectorizer_desc = TfidfVectorizer()
description_vectors = vectorizer_desc.fit_transform(data_arco['Descripción'])

In [22]:
# Selected features
selected_features = ['Emprendedores', 'Microempresa', 'Pequeña Empresa', 'Mediana Empresa', 'Gobierno ', 'Academia', 'Personas naturales']

# Fill missing values with empty strings
for feature in selected_features:
    data_arco[feature] = data_arco[feature].fillna('')

# combining all the 5 selected features - Con todos los tipos de usuarios
combined_features = data_arco['Emprendedores'] + ' ' + data_arco['Microempresa'] + ' ' + data_arco['Pequeña Empresa'] + ' ' + data_arco['Mediana Empresa'] + ' ' + data_arco['Gobierno '] + ' ' + data_arco['Academia'] + ' ' + data_arco['Personas naturales'] 

# Vectorize the combined features
vectorizer_features = TfidfVectorizer()
feature_vectors = vectorizer_features.fit_transform(combined_features)

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

# Description similarity matrix
description_similarity = cosine_similarity(description_vectors, description_vectors)

# Features similarity matrix
features_similarity = cosine_similarity(feature_vectors, feature_vectors)

In [24]:
# Define weights
weight_desc = 0.7
weight_features = 0.3

# Ensure both matrices are in the range [0, 1]
# If necessary, normalize the matrices
# For cosine similarity, values are already between -1 and 1, but we can adjust them to [0, 1]
description_similarity = (description_similarity + 1) / 2
features_similarity = (features_similarity + 1) / 2

# Combine the similarity matrices
combined_sim = (weight_desc * description_similarity) + (weight_features * features_similarity)

In [26]:
from difflib import SequenceMatcher

def find_closest_name(query, names):
    max_ratio = 0
    closest_name = ""
    for name in names:
        ratio = SequenceMatcher(None, query, name).ratio()
        if ratio > max_ratio:
            max_ratio = ratio
            closest_name = name
    return closest_name

def recommend_instruments(input_name, data_arco, combined_sim, top_n=5):
    # Find the closest instrument name
    closest_name = find_closest_name(input_name, data_arco['Nombre del Instrumento'])
    # Get the index of the selected instrument
    selected_index = data_arco[data_arco['Nombre del Instrumento'] == closest_name].index[0]
    # Get similarity scores
    sim_scores = list(enumerate(combined_sim[selected_index]))
    # Sort the scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Exclude the selected instrument
    sim_scores = [x for x in sim_scores if x[0] != selected_index]
    # Get top N indices
    top_indices = [i[0] for i in sim_scores[:top_n]]
    # Return recommended instruments
    return data_arco.iloc[top_indices][['Código', 'Nombre del Instrumento', 'Sector', 'Entidad']]

In [28]:
# Example usage
recommendations = recommend_instruments("Fábricas de Productividad y Sostenibilidad - Huila", data_arco, combined_sim, top_n=20)
print(recommendations)

    Código                             Nombre del Instrumento    Sector  \
9     3419  CELEBRACIÓN DEL DÍA DEL CAMPESINO DEL MUNICIPI...       NaN   
3     3321                                          CEmprende  Comercio   
4     3403  VERSIÓN 184 DE LA FERIA GANADERA, MERCADO GANA...       NaN   
7     3415    feria internacional de cafe cacao y agroturismo       NaN   
10    3428  Convocatoria Concertación Proyectos y Estimulo...       NaN   
5     3407  FERIA EQUINA GRADO B EN SU VERSIÓN 185, MERCAD...       NaN   
11    3760                                       Convocatoria       NaN   
12    3762  Gestión de la Información Estadística y Cartog...       NaN   
2     3308                                       Expovivienda       NaN   
8     3416  CONVOCATORIA PARA EL SECTOR TURISMO DEL DEPART...       NaN   
1     3304                 Cursos virtuales de Emprendimiento       NaN   
6     3410  Sistema de Información Turístico y Cultural de...       NaN   

                        

In [29]:
# Directorio DNP
recommendations.to_excel('C:/Users/supegui/Documents/Metodología ArCo/Python/Excel/recomendaciones_arco_huila.xlsx', index=False)

# Directorio remoto
#recommendations.to_excel('C:/Backup - Archivos de trabajo/DNP/Automatización ArCo/Excel/recomendaciones.xlsx', index=False)